In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [66]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from csv import writer

# Airbnb regex

In [3]:
import re
import requests

In [105]:
file_name1 = 'Mining/bangli-18-Januari.xlsx'
# file_name2 = 'Mining/jembrana-negara-16-Januari.xlsx'
# file_name3 = 'Mining/jembrana-negara.xlsx'
df_airbnb1 = pd.read_excel(file_name1)
# df_airbnb2 = pd.read_excel(file_name2)
# df_airbnb3 = pd.read_excel(file_name3)

In [22]:
df_airbnb3 = df_airbnb3.drop(columns=['title'])

In [106]:
df_airbnb1

,link
0,https://airbnb.co.id/s/Apuan--Susut--Bangli/ho...
1,https://airbnb.co.id/rooms/49255906?adults=1&c...
2,https://airbnb.co.id/rooms/603975875474169407?...
3,https://airbnb.co.id/rooms/48383754?adults=1&c...
4,https://airbnb.co.id/rooms/40050774?adults=1&c...
...,...
19962,https://airbnb.co.id/rooms/632449053092568933?...
19963,https://airbnb.co.id/rooms/42283165?adults=1&c...
19964,https://airbnb.co.id/rooms/46286372?adults=1&c...
19965,https://airbnb.co.id/rooms/632454465364180392?...


In [107]:
# df_total = pd.concat([df_airbnb1, df_airbnb2], ignore_index=True)
df_total = df_airbnb1

In [108]:
df_total.nunique()

link    2158
dtype: int64

In [109]:
df_total.drop_duplicates(subset ="link", keep = 'first', inplace = True)
df_total

,link
0,https://airbnb.co.id/s/Apuan--Susut--Bangli/ho...
1,https://airbnb.co.id/rooms/49255906?adults=1&c...
2,https://airbnb.co.id/rooms/603975875474169407?...
3,https://airbnb.co.id/rooms/48383754?adults=1&c...
4,https://airbnb.co.id/rooms/40050774?adults=1&c...
...,...
19824,https://airbnb.co.id/s/Belandingan--Kintamani-...
19849,https://airbnb.co.id/s/Belandingan--Kintamani-...
19877,https://airbnb.co.id/s/Belandingan--Kintamani-...
19886,https://airbnb.co.id/s/Belandingan--Kintamani-...


In [110]:
for item in df_total['link']:
    try:
        match = re.search(r"https://airbnb\.co\.id/rooms/\d+", item)
        result = match.group()
        df_total['link'] = df_total['link'].replace(item, result)
    except AttributeError:
        df_total['link'] = df_total['link'].replace(item, np.nan)
    

In [111]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2158 entries, 0 to 19908
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    1229 non-null   object
dtypes: object(1)
memory usage: 33.7+ KB


In [112]:
df_total = df_total.dropna()
df_total.reset_index(drop=True, inplace=True)
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1229 entries, 0 to 1228
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    1229 non-null   object
dtypes: object(1)
memory usage: 9.7+ KB


In [113]:
list_total = df_total['link'].tolist()

In [114]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
link = list_total[0]
page = requests.get(link, headers=headers)
soup = BeautifulSoup(page.content, 'html5lib')
soup.title

<title>Kamar bulan madu dengan pemandangan hutan di jantung Ubud - Resor untuk Disewakan di Kecamatan Ubud, Bali, Indonesia - Airbnb</title>

In [115]:
df_total = pd.DataFrame (columns = ['link','judul','alamat','kategori','tahun','tuan_rumah'])

In [98]:
list_total[600]

'https://airbnb.co.id/rooms/567710730696662278'

In [116]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
for i in range(0,len(list_total)):
    try:
        list_clean = list()
        url = list_total[i]
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, 'html5lib')
        title = str(soup.title)
        # ================================ Judul, Alamat, Kategori ================================
        result = re.sub(r"<title>|</title>", "", title)
        match1 = re.search(r"(.*) - (.*) - (.*)", result)
        part1 = match1.group(1) # Judul
        # print(part1)
        part2 = match1.group(2) # Alamat dan kategori
        alamat = re.sub(r".*di", "", part2)
        alamat = re.sub(r"^\s*", "", alamat)
        # print(alamat)
        kategori = re.search(r"^\w+", part2).group()
        # print(kategori)
        
        text = soup.get_text()
        # ================================ Tahun ================================
        match = re.search(r'{"__typename":"HostProfileSection","title":"(.*?)","subtitle":"(.*?)"', text)
        tahun = match.group(2)
        # ================================ Tuan Rumah ================================
        match = re.search(r'{"__typename":"PdpOverviewSection","title":"(.*?)"', text)
        tuan_rumah = match.group(1)
        # Penanda index dan judul
        print(f'No : {i}, {part1}')

        list_clean.append([url,part1,alamat,kategori,tahun,tuan_rumah])
        df = pd.DataFrame (list_clean, columns = ['link','judul','alamat','kategori','tahun','tuan_rumah'])
        df_total = pd.concat([df_total, df], ignore_index=True)
    except AttributeError:
        print(f'No : {i}, ERROR')
        continue


No : 0, Kamar bulan madu dengan pemandangan hutan di jantung Ubud
No : 1, Kamar Studio di Gianyar
No : 2, Kamar Deluxe dengan Balkon Pribadi di Ubud
No : 3, Surga kecil di jantung ubud
No : 4, Kamar di Ubud
No : 5, Vila cantik baru di Ubud
No : 6, Villa Nyaman Menawan Dekat dengan Semua Tempat Wisata di Ubud
No : 7, Bed and breakfast di Ubud Tengah, kolam renang, AC
No : 8, 360 derajat Pemandangan Matahari Terbit &amp; Matahari Terbenam di Ubud
No : 9, Resor Rimba Tropis di Payangan Ubud
No : 10, Superior Suite Ubud City Centre
No : 11, Anggrek 1 - Bali yang sesungguhnya di kemewahan barat!
No : 12, Kamar Bergaya di Ubud, Permata Tersembunyi yang Unik
No : 13, Sari Uma Guesthouse 4
No : 14, Kamar balkon dengan hammock luar ruangan di Ubud.
No : 15, BARU! Suite yang damai dengan Pemandangan Rimba
No : 16, Kamar Menakjubkan Dekat Pasar Ubud - Kolam Luar Ruangan!!
No : 17, Kamar unggulan di cendana
No : 18, Puri Payogan Villa - Ubud ( Deluxe Pool Access )
No : 19, Tempat Tidur Super King 

In [117]:
df_total

,link,judul,alamat,kategori,tahun,tuan_rumah
0,https://airbnb.co.id/rooms/49255906,Kamar bulan madu dengan pemandangan hutan di j...,"Kecamatan Ubud, Bali, Indonesia",Resor,Bergabung sejak Maret 2021,Kamar pribadi di resor. Tuan rumah: Ni Putu
1,https://airbnb.co.id/rooms/603975875474169407,Kamar Studio di Gianyar,"Kecamatan Blahbatuh, Bali, Indonesia",Hotel,Bergabung sejak Agustus 2017,Kamar di hotel. Tuan rumah: Arista
2,https://airbnb.co.id/rooms/48383754,Kamar Deluxe dengan Balkon Pribadi di Ubud,"Tampaksiring, Bali, Indonesia",Resor,Bergabung sejak Agustus 2015,Kamar pribadi di resor. Tuan rumah: Island Escape
3,https://airbnb.co.id/rooms/40050774,Surga kecil di jantung ubud,"Kecamatan Ubud, Bali, Indonesia",Hotel,Bergabung sejak Agustus 2019,Kamar di hotel. Tuan rumah: Dewa
4,https://airbnb.co.id/rooms/620491660432846424,Kamar di Ubud,"Kecamatan Ubud, Bali, Indonesia",Bed,Bergabung sejak Januari 2016,Kamar pribadi di bed and breakfast. Tuan rumah...
...,...,...,...,...,...,...
1211,https://airbnb.co.id/rooms/670144271000313382,Laksmi homestay akan menyelesaikan pengalaman ...,"Kecamatan Kubutambahan, Bali, Indonesia",Bed,Bergabung sejak Desember 2017,Kamar pribadi di bed and breakfast. Tuan rumah...
1212,https://airbnb.co.id/rooms/38759181,laksmi homestay air sanih,"Kabupaten Buleleng, Indonesia | Airbnb",Kabupaten,Bergabung sejak Desember 2017,Kamar di bed and breakfast. Tuan rumah: Laksmi
1213,https://airbnb.co.id/rooms/617000470951547422,Vila &amp; Spa Butik Griya Bali Surga - KAMAR ...,"Kecamatan Kubutambahan, Bali, Indonesia",Bed,Bergabung sejak Oktober 2021,Kamar pribadi di bed and breakfast. Tuan rumah...
1214,https://airbnb.co.id/rooms/616965281854565974,Vila &amp; Spa Butik Griya Bali Surga - KAMAR ...,"Kecamatan Kubutambahan, Bali, Indonesia",Bed,Bergabung sejak Oktober 2021,Kamar pribadi di bed and breakfast. Tuan rumah...


In [118]:
df_total.to_excel('bangli-airbnb.xlsx', index=False)

In [103]:

from selenium import webdriver
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0;Win64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}
url = list_total[0]
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html5lib')
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)


C:\Users\Nara\AppData\Local\Temp\ipykernel_12052\579584000.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
